In [31]:
import pandas as pd

X = pd.read_csv('/content/data/train.csv')

y = X['price_range']
X.drop(['price_range'], axis=1, inplace=True)

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [32]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

gb_params = {
    'learning_rate' : [0.01, 0.2, 0.3],
    'n_estimators' : [100, 200, 300],
    'max_depth' : [3, 5, 6],
    'subsample' : [0.7, 0.9, 1.0],
    'colsample_bytree' : [0.7, 0.9, 1.0],
    'gamma' : [0.1, 0.2, 0.3],
}

model = XGBClassifier()

grid_search = GridSearchCV(estimator=model,
                           param_grid=gb_params,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.9, 1.0],
                         'gamma': [0.1, 0.2, 0.3],
                         'learning_rate': [0.01, 0.2, 0.3],
                         'max_depth': [3, 5, 6],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.7, 0.9, 1.0]},
             scoring='accuracy', verbose=1)

In [33]:
import joblib
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

model = grid_search.best_estimator_

joblib.dump(model, 'XGBClassifier.pkl')

y_preds = model.predict(X_valid)
accuracy_score(y_valid, y_preds)

0.9225

In [35]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_sc = scaler.fit_transform(X_train)

X_scaled = pd.DataFrame(X_sc)
X_scaled.columns = X_train.columns
X_valid_scaled = pd.DataFrame(scaler.transform(X_valid))
X_valid_scaled.columns = X_valid.columns

In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

lr_params ={
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 300],
    'l1_ratio': [0.5],
    'class_weight': [None, 'balanced']
}

model = LogisticRegression()

grid_search = GridSearchCV(model, lr_params, cv=5,
                           scoring='accuracy', n_jobs=-1,
                           verbose=2)

grid_search.fit(X_scaled, y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
240 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1169, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "

GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100],
                         'class_weight': [None, 'balanced'], 'l1_ratio': [0.5],
                         'max_iter': [100, 200, 300],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['liblinear', 'saga']},
             scoring='accuracy', verbose=2)

In [37]:
from sklearn.metrics import accuracy_score
model = grid_search.best_estimator_

preds = model.predict(X_valid_scaled)
print(classification_report(y_valid, preds))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        95
           1       0.98      0.93      0.96       104
           2       0.96      0.94      0.95       107
           3       0.96      0.99      0.97        94

    accuracy                           0.96       400
   macro avg       0.97      0.97      0.97       400
weighted avg       0.97      0.96      0.96       400



In [38]:
import joblib

joblib.dump(model, 'LogisticRegression.pkl')

['LogisticRegression.pkl']